In [1]:
import csv, sys, os, math
import pandas as pd
import numpy as np
from numpy import argmax
import random
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,auc  
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import pyplot
from functools import reduce
from pathlib import Path
import lightgbm as lgb

In [2]:
workdir = os.path.abspath('.')
exp_data = pd.read_csv('%s/BRCA.sampleMap_HiSeqV2'%workdir,sep='\t')
gene_name = exp_data['sample']
exp_data = exp_data.drop(labels = 'sample',axis=1).transpose()
exp_data.columns = gene_name
ylable= pd.DataFrame(pd.DataFrame(exp_data.index)[0].str.split('-').str[-1])
exp_data['ylable'] = np.where(ylable[0].str.contains("0"), 1, 0)
exp_data.to_csv('%s/BRCA.txt'%workdir,sep='\t',index='sample')

x = exp_data.iloc[:,0:20531]
x = x.drop(columns = 'ylable')
y = exp_data.iloc[:,-1].astype(int)

In [3]:
%%time
importance_list = []
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2,random_state = 32, stratify=y)

params = {
            'objective':'binary',
            #max_depth=-1,
            'learning_rate': 0.005, 
            #min_child_samples=3, 
            #random_state=2023,
            #n_estimators=5000, 
            #silent = False,
            'subsample':0.5, 
            'colsample_bytree':0.75}
model = lgb.LGBMClassifier(**params)
model.fit(x_train,y_train)

importance = model.feature_importances_
importance_list.append(importance)

CPU times: total: 8min 38s
Wall time: 51.4 s


from lightgbm import plot_importance
plot_importance(model,importance_type="gain")
plot_importance(model, importance_type="split")

feature_imp = pd.DataFrame({'Value':importance,'Feature':x.columns})

feature_imp

model,importance_type="gain"

In [4]:
df_importance = pd.DataFrame(importance_list).fillna(0)
df_importance.columns = x.columns
df_result = df_importance.mean(axis =0).sort_values(ascending= 0)
max_importance = df_result[0]

In [5]:
#十次cv分别每次实验的特征重要性数值（无序）
df_importance

sample  ARHGEF10L  HIF3A  RNF17  RNF10  RNF11  RNF13  GTF2IP1  REM1  MTVR2  \
0               2      2      0      0      0      0        1     0      1   

sample  RTN4RL2  ...  TULP2  NPY5R  GNGT2  GNGT1  TULP3  PTRF  BCL6B  GSTK1  \
0             0  ...      0      0      0      0      0     0      1      0   

sample  SELP  SELS  
0          0     0  

[1 rows x 20530 columns]

In [6]:
#最终的特征重要性（有序）
df_result

sample
SPRY2       74.0
SCN4B       66.0
ADAMTS5     64.0
THRB        60.0
MMP11       41.0
            ... 
WWP2         0.0
ROBO2        0.0
MGC16703     0.0
PCDHB1       0.0
SELS         0.0
Length: 20530, dtype: float64

In [7]:
#%pprint
alist = list(df_result.index)#有序版
name=['LightGBM']
name2=range(1,len(df_result.index)+1)
test=pd.DataFrame(columns=name,index=name2,data=alist)
print(test)

       LightGBM
1         SPRY2
2         SCN4B
3       ADAMTS5
4          THRB
5         MMP11
...         ...
20526      WWP2
20527     ROBO2
20528  MGC16703
20529    PCDHB1
20530      SELS

[20530 rows x 1 columns]


In [8]:
test.to_csv('lgb特征排序结果.csv')